# Modules

In [ ]:
%matplotlib widget
import os
import glob
import psfex
import pickle
import sncosmo
import numpy as np
import pandas as pd
import iminuit as im
import astropy as ap
import ipywidgets as ipw
import matplotlib.pyplot as plt
import ztfquery
import ztfimg
import ztflc

from ztfimg import dao
from ztfimg import image
from ztflc import fitter
from ztflc import diffdata
from astropy.io import fits
from ztfquery import marshal
from astrobject import photometry
from ztflc import forcephotometry
from scipy.stats import multivariate_normal
from sncosmo.models import Source, Model, get_source
from mpl_toolkits.axes_grid1 import make_axes_locatable
from sncosmo import get_bandpass, get_magsystem
from sncosmo.photdata import photometric_data

In [2]:
root = "../Data/"
psfoutcat = root + 'config_ZOGY/BlackGEM_default/psfex_out.cat'
mask = root + "ztf_20190124095417_000403_zr_c01_o_q1_mskimg.fits"
psfcat = root + "ztf_20190124095417_000403_zr_c01_o_q1_psfcat.fits"
sciimg = root + "ztf_20190124095417_000403_zr_c01_o_q1_sciimg.fits"
diffpsfpath = root + 'ztf_20190124095417_000403_zr_c01_o_q1_diffimgpsf.fits'
psfcent = root + "ztf_20190124095417_000403_zr_c01_o_q1_sciimgdaopsfcent.fits"
diffimgpath = root + 'ztf_20190124095417_000403_zr_c01_o_q1_scimrefdiffimg.fits.fz'

hdul = fits.open(sciimg)
pdul = fits.open(psfcat)
cdul = fits.open(psfcent)

cdata = cdul[0].data
header = hdul[0].header

#df_psfcat = pd.DataFrame(pdul[1].data)
#x_list, y_list = df_psfcat['xpos'], df_psfcat['ypos']

df_outcat = pd.read_csv(psfoutcat, delim_whitespace=True)
x_list, y_list = df_outcat[['x', 'y']].values.T
xy_list = df_outcat[['x', 'y']].values

zimg = photometry.Image(sciimg, background=0)
ra_list, dec_list = np.asarray([zimg.pixel_to_coords(x, y) for x, y in xy_list]).T

z = image.ScienceImage(sciimg, mask)
z.load_source_background()
z.set_catalog(df_outcat, 'outcat')

row = z.header["NAXIS1"]
col = z.header["NAXIS2"]

plt.close()
ax = z.show('dataclean')
ax.scatter(x_list, y_list, color='C1', marker='+')

In [9]:
diffimg = '../Data/ztf_20190124095417_000403_zr_c01_o_q1_scimrefdiffimg.fits.fz'
diffpsf = '../Data/ztf_20190124095417_000403_zr_c01_o_q1_diffimgpsf.fits'
ind = 100
fit_obj = ztflc.diffdata.DiffData(diffimg,
                                  diffpsf,
                                  (ra_list[ind],
                                   dec_list[ind]))

In [10]:
fit_obj.fit_flux()

{'sigma': 7.001478769590302,
 'sigma.err': 0.1399685270627203,
 'ampl': 9.325205108081073,
 'ampl.err': 29.94603771622619,
 'fval': 4206.3381759299555,
 'chi2': 625.01434,
 'chi2dof': 1.0032332957651986}

In [9]:
diffimg_data = fits.getdata(diffimg)
diffpsf_data = fits.getdata(diffpsf)